jupyter-lab

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math


In [3]:
Data = pd.read_excel(r'C:\Users\Mehdi\Desktop\AllComp.xlsx')
Data_Results = pd.read_excel(r'C:\Users\Mehdi\Desktop\1.xlsx')
FAH_CU = Data_Results[(Data_Results.form == "FAH")]
Data_Numpy = FAH_CU.to_numpy()
FAH_CU2 = pd.DataFrame(Data_Numpy)

GOR Estimator

In [7]:
xi = []
yi = []
zi = []
Ki = []
GOR = []
diff = []
#Data without n2, co2, h2s
Data2 = Data.iloc[3:, :]
Liq_Dens = 62.4 * (141.5 / (FAH_CU2.loc[:, 7] + 131.5)) 
MW = FAH_CU2.loc[:, 24]
GOR_Real = FAH_CU2.loc[:, 8]
GOR_Real = GOR_Real.to_numpy()
counter = 0
for i in Data.columns:
    Data3 = Data2[i].dropna()
    if i%4 == 1:
        xi.append(Data3)
    elif i%4 == 2:
        yi.append(Data3)
    elif i%4 == 3:
        zi.append(Data3)
        counter += 1      
for i in range(0,counter):
    b, a = np.polyfit(xi[i], yi[i], deg=1)
    GOR.append(379.5 * Liq_Dens[i] * 5.615 / (-b * MW[i]))  #lb/ft3
    diff.append(abs((GOR[i] - GOR_Real[i])*100/GOR_Real[i]))

print(GOR)
print(GOR_Real)
print(diff)

[2101.304188741372, 2095.296110875707, 1950.388708939189, 1561.377783175239, 1840.8718750030516, 5809.2793099629935, 1683.4662248896261, 1463.567734019833, 1762.5357180975363, 1877.5550919706034, 1591.285778553399, 1955.6050047585281, 880.7033616438486, 984.3119549936215, 1200.2281032915366, 979.3655112593386, -1300.0810372485919, 21828.560757911833, 3437.3833910137187, 1932.7021793723118, 2933.2738795208143, 2048.8624634131866, 596.4664733269667, 693.3755728731079, 847.1930805753852, 580.2597811747206, 638.0582024266483, 578.0487736354638, 603.0425393721657, 1868.9174099428758, 2454.9058278369794, 3049.8327832843224, 1566.9977231130874, 2022.019259611172, 2206.529507372403, 2100.2137210779547, -667.9570899077704, -1218.200653273869]
[3548.32 1730.0 2492.69 2023.0 2115.0 2230.0 2407.0 2084.0 2329.0 1922.8
 1190.0 1583.12 1777.85 2048.0 2098.0 1170.26 1577.45 3539.08 1805.68
 1499.17 1582.25 1998.39 1396.34 1328.93 1527.62 1180.49 1579.52 1474.5
 1454.65 1916.15 1678.28 1995.91 1090.06 

Hoffman plot

In [4]:
Tb = [201.27, 332.51, 416.25, 470.78, 491.08, 542.12, 556.92]
Tc = [343.33, 549.92, 666.06, 734.46, 765.46, 829.1, 845.8]
Pc = [666.4, 706.5, 616, 527.9, 550.6, 490.4, 488.6]
T = 294.3 +460
Psc = 14.7
Psep = 14.7
F = []
Dist = []

#Hoffman factor
for i in range(0, len(Tb)):
    F.append(((1/Tb[i])-(1/T)) * (np.log10(Pc[i]/Psc)) / ((1/Tb[i])-(1/Tc[i])))


for i in range(0,counter):
    Ki = yi[i][0:7]/xi[i][0:7]
    #Plot log(Ki*Psep) Vs Fi
    #plt.figure()
    prod = np.dot(Ki, Psep)
    logprod = np.log10(prod)
    #plt.scatter(F, logprod)
    m, c = np.polyfit(F, np.log10(prod), deg=1)
    xseq = np.linspace(0, 3, num=100)
    Dist.append(np.sum(abs(np.dot(m, F) - logprod + c) / math.sqrt(m**2 + 1)))
    #plt.plot(xseq, c + m * xseq)
    #plt.show()
    
print('Distance: ', Dist)

Distance:  [0.250216728998782, 0.250216728998782, 0.6364984427121599, 0.8093388955857574, 0.5103156474611831, 0.10617078077840261, 0.6909351548604232, 0.5283472472695802, 0.8215532714150005, 0.6603160017899314, 0.6855546060503819, 1.4032427494222357, 0.42425429884243426, 0.3190166759837152, 0.6504110082874737, 0.7860519587575115, 0.2738364368989459, 0.18033977728221393, 0.8656942325727438, 0.7294924234574626, 0.7964878309566242, 1.270027294707794, 0.30377361246463463, 0.5335928321418065, 0.9923594676796654, 0.22766181899842727, 0.49647921782466453, 0.3760294378601974, 0.3760294378601974, 0.3760294378601974, 0.2499425594681558, 0.8786031890447901, 0.270577027718791, 0.44251525906372213, 0.8169043134513221, 0.5373783445680467, 0.5373783445680467, 0.28108914474958047]


Campbell-Buckley

In [5]:
Dist2 = []
xval = np.power(Tc, 2)/100000
for i in range(0,counter):
    Ki = yi[i][0:7]/xi[i][0:7]
    #plt.figure()
    logK = logprod = np.log10(Ki)
    #plt.scatter(xval, logK)
    m, c = np.polyfit(xval, logK, deg=1)
    xseq = np.linspace(0, 8, num=100)
    Dist2.append(np.sum(abs(np.dot(m, xval) - logK + c) / math.sqrt(m**2 + 1)))
    #plt.plot(xseq, c + m * xseq)
    
print(Dist2)

[0.30193383009799235, 0.30193383009799235, 1.1655116204830391, 1.924331552296506, 1.1948240085234014, 0.1105492193284572, 1.5432227971569412, 1.2600672127452583, 1.7746331076176776, 1.480267387958397, 1.5119818923098574, 2.7965317887009626, 0.5244677737537758, 0.36848394586463046, 1.0886244958677276, 1.403109891082761, 0.27705188024235117, 0.14811676280603678, 2.009000013274023, 1.0066998264853386, 1.8497364738060513, 2.5766087974190537, 0.8564792926061708, 1.3552514552553403, 2.232580285921803, 0.6461244053837614, 1.354685523717695, 1.020131396814526, 1.020131396814526, 1.020131396814526, 0.2473098007805743, 1.5817016322339814, 0.28542028353160176, 0.7359200772816396, 1.7517297216090393, 0.6896616393066276, 0.6896616393066276, 0.34304202240181225]
